# Datasets

Datasets consist of complete combinatorial landscapes to visualize and work with, as well as the data from which the were inferred. Both inference of the complete landscape and calculation of the coordinates of the visualization are precomputed to provide rapid access to the different layers of interest.

In [1]:
# Import required libraries
import numpy as np

from gpmap.src.datasets import DataSet, list_available_datasets
from gpmap.src.inference import VCregression, SeqDEFT

## How to load a built-in dataset

We include a series of datasets that are used throughout the documentation for demonstration of the different applications and are directly accessible after installation of the library for any user.
The list of built-in datasets can be easily shown as follows

In [2]:
list_available_datasets()

['5ss', 'f1u', 'test', 'dmsc', 'gb1', 'smn1', 'serine', 'trna', 'pard']

### How to access combinatorial landscape values

And one can easily load one of those datasets as illustrated in some previous tutorials, and all of them should contain at least a `landscape` attribute containing the phenotype associated to each possible genotype

In [3]:
gb1 = DataSet('gb1')
gb1.landscape

,y
seq,
AAAA,0.296301
AAAC,-2.713474
AAAD,-2.912992
AAAE,-4.548719
AAAF,-3.276738
...,...
YYYS,-4.662925
YYYT,-3.223102
YYYV,-3.001718


### How to access the processed data in experimental datasets

If the landscape was obtained from experimental data, then it also has a `data` attribute that includes the measurement `y` and, if available, its uncertainty `y_var`. 
The data may not necessarily include measurements for every possible sequence, as in this case, in which about ~10000 sequences were not experimentally measured

In [4]:
gb1.data

,y,y_var
sequence,,
AAAA,0.460831,0.046009
AAAG,-2.192261,0.255906
AAAH,-4.728306,2.064530
AAAI,-4.338842,2.095252
AAAL,-2.326240,0.087518
...,...,...
YYYS,-5.269987,0.291090
YYYT,-3.821426,0.074489
YYYV,-3.143536,0.074682


### How to access the a dataset visualization

For built-in datasets, we also provide the pre-calculated coordinates of the visualization, the `DataFrame` connecting sequences separated by single point mutations and the relaxation times associated to each of the diffusion axes in the attributes `nodes`, `edges` and `relaxation_times`

In [5]:
gb1.nodes

,1,2,3,4,5,6,7,8,9,10,function,stationary_freq
AAAA,-0.270938,-0.944304,-0.227171,0.744803,0.059077,-0.077512,-0.477853,0.174491,0.015944,0.052664,0.296301,1.067767e-04
AAAC,0.033789,-0.232603,-0.271458,0.576487,0.035619,0.087608,0.590118,-0.249005,-0.087750,-0.110291,-2.713474,4.954648e-06
AAAD,-0.020398,-0.127749,-0.174455,0.347843,0.142684,0.208679,0.590025,0.160819,0.354397,0.676487,-2.912992,4.042194e-06
AAAE,-0.001018,-0.138712,-0.183161,0.340728,0.121067,0.157871,0.436407,0.195630,0.211100,0.364298,-4.548719,7.619345e-07
AAAF,0.149717,-0.156524,-0.239304,0.386243,0.103285,0.107756,0.302406,0.051575,0.171278,0.226772,-3.276738,2.789084e-06
...,...,...,...,...,...,...,...,...,...,...,...,...
YYYS,0.073880,0.038075,-0.097751,0.156184,0.056463,0.074291,0.262512,0.019037,0.144439,0.172686,-4.662925,6.781399e-07
YYYT,-0.091125,0.213370,0.256403,0.246274,-0.086279,0.026923,0.217086,0.102111,0.593257,0.003682,-3.223102,2.945947e-06
YYYV,0.016488,0.195242,0.216320,0.035269,0.306726,0.025334,0.217759,-0.028542,-0.038378,0.148356,-3.001718,3.692393e-06
YYYW,0.134072,0.114107,-0.043856,0.011092,0.076565,0.109907,0.261274,0.108909,0.180371,0.365348,-4.723318,6.376209e-07


In [6]:
gb1.edges

,i,j
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
6079995,159996,159998
6079996,159996,159999
6079997,159997,159998
6079998,159997,159999


In [7]:
gb1.relaxation_times

,k,decay_rates,relaxation_time
0,1,2.554843,0.391413
1,2,3.566862,0.280359
2,3,4.926568,0.202981
3,4,5.023657,0.199058
4,5,5.303026,0.188572
5,6,5.635594,0.177444
6,7,6.294868,0.158860
7,8,6.543588,0.152821
8,9,6.741685,0.148331
9,10,7.000798,0.142841


## How to build new datasets

We also provide functionality to create new datasets and store them in the local copy of your library for easier access. Lets build new datasets from simulated data

In [8]:
np.random.seed(0)
lambdas = np.array([10, 2, 0.5, 0.1, 0.02, 0])
model = VCregression(seq_length=5, alphabet_type='dna', lambdas=lambdas)
data = model.simulate(p_missing=0.2, sigma=0.1).drop('y_true', axis=1).dropna()
data

,y,y_var
AAAAA,0.287342,0.01
AAAAC,0.272354,0.01
AAAAG,-0.049616,0.01
AAAAT,-0.350486,0.01
AAACA,0.130318,0.01
...,...,...
TTTGA,-0.015608,0.01
TTTGC,-0.136017,0.01
TTTGG,0.169651,0.01
TTTGT,-0.499220,0.01


The method `build` will use some default values to run Variance Component regression and compute visualization coordinates automatically, but may not be the best choice for any particular dataset.

In [9]:
test = DataSet('test', data=data)
test.build()

100%|██████████| 100/100 [00:05<00:00, 19.62it/s]


We can now re-load the dataset from disk and verify that it contains the visualization attributes

> Note that reinstalling the library will erase the newly created `DataSet`s

In [10]:
test = DataSet('test')
test.nodes

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,function,stationary_freq
AAAAA,1.506422,-1.399447,-0.631288,1.815978,2.508739,0.230666,-0.304176,-0.240791,-0.138208,0.052919,...,-0.071852,0.957932,0.168867,0.331075,0.130200,0.122967,-0.021842,-0.065675,0.210848,7.040323e-04
AAAAC,2.007200,-0.231637,-0.229503,2.026901,2.116516,-0.061370,-0.687321,-0.057379,-0.097677,-0.418981,...,0.131944,-0.356953,-0.021197,0.031237,-0.053747,0.352218,-0.111756,-0.249799,0.239703,1.022874e-03
AAAAG,1.453408,-0.021478,0.249886,2.642317,0.781247,-0.345954,0.975222,-0.016601,-0.401115,-1.012593,...,-0.375792,-0.616726,0.818233,0.380967,1.136530,0.462220,-0.085668,0.028635,-0.024662,3.337899e-05
AAAAT,1.184685,-0.062999,-0.194603,1.766047,1.215702,-0.032267,0.188691,-0.026001,-0.087853,-0.357104,...,-0.686679,0.244378,0.066704,0.230745,0.362764,0.764556,-0.010949,0.050858,-0.298907,9.584686e-07
AAACA,1.285799,-0.816036,-0.190119,2.017239,1.250325,-0.018434,-0.054812,0.037966,-0.059010,-0.535600,...,-0.105837,1.014249,-0.106504,0.295286,0.033131,0.084332,-0.016234,-0.005161,0.092795,1.527064e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGT,-0.998509,-0.298003,0.036903,0.174085,-0.283209,0.087098,0.089608,1.045328,0.918049,0.532736,...,0.110641,0.100165,-0.186232,0.631784,0.097940,-0.052740,1.000785,0.457957,-0.458989,1.206530e-07
TTTTA,-0.971021,-0.143357,1.638933,-0.347183,0.387400,0.377610,-0.015073,1.361348,0.380824,0.338251,...,0.136856,0.043304,0.148290,0.301352,-0.125303,-0.027340,-0.287078,0.032937,-0.263644,1.512992e-06
TTTTC,-0.833931,0.670250,2.619121,-0.518760,0.808557,0.527004,-0.131234,2.433294,-0.216354,0.274174,...,0.272212,0.063191,-0.490595,-0.178522,0.370356,-0.057199,-0.457634,0.218082,0.022313,6.131714e-05
TTTTG,-0.978880,0.783332,1.139558,0.687252,-0.320269,0.408076,0.181198,2.276325,2.346162,1.041281,...,0.271400,-0.236592,1.074526,0.197961,-0.638422,0.086949,0.113400,0.267318,-0.094331,1.354474e-05
